In [1]:
# Enabling print for all lines
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Checking the working directory
import os
os.getcwd()

'C:\\Users\\kalya\\Python\\NLP'

In [5]:
# Importing libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Stopword removal
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import random
random.seed(10)
print(random.random())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kalya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

0.5714025946899135


In [6]:
# !pip install gensim
# !pip install pyLDAvis
# !python -m spacy download en
nlp = spacy.load('en_core_web_sm')

**Prepare Stopwords**

In [7]:
# NLTK Stop words
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

**Importing 20-Newsgroups data**

In [8]:
# Dataset contains about 11k newsgroups posts from 20 different topics, available as newsgroups.json
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
df.shape
df.target_names.unique()
df.head()

(11314, 3)

array(['rec.autos', 'comp.sys.mac.hardware', 'comp.graphics', 'sci.space',
       'talk.politics.guns', 'sci.med', 'comp.sys.ibm.pc.hardware',
       'comp.os.ms-windows.misc', 'rec.motorcycles', 'talk.religion.misc',
       'misc.forsale', 'alt.atheism', 'sci.electronics', 'comp.windows.x',
       'rec.sport.hockey', 'rec.sport.baseball', 'soc.religion.christian',
       'talk.politics.mideast', 'talk.politics.misc', 'sci.crypt'],
      dtype=object)

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


**Remove email and newline characters**

In [9]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


**Tokenization and Text clean-up**

In [10]:
# Need to break down each sentence into a list of words through tokenization
def sent_to_words(sentences):
    for sentence in sentences:
        # Gensim’s simple_preprocess() is great for tokenization and deacc=True removes punctuations 
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))
print(data_words[:1], end='')

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]

**Creating Bigram and Trigram Models**

In [11]:
# Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams
"Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring"

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]], end='')

'Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring'

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']

**Remove Stopwords, Make Bigrams and Lemmatize**

- Lemmatization is nothing but converting a word to its root word
- For example: the lemma of the word ‘machines’ is ‘machine’
- Likewise, 'walking' –> 'walk', 'mice' –> 'mouse'

In [12]:
# Define functions for stopwords removal, making bigrams, trigrams and lemmatization sequentially
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1], end=',')

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']],

**Create the Dictionary and Corpus needed for Topic Modeling**

In [14]:
# The two main inputs to the LDA topic model are the dictionary and the corpus

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1], end='')

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)]]

- Gensim creates a unique id for each word in the document
- The produced corpus shown above is a mapping of (word_id, word_frequency)
- (0, 1) above implies, word id 0 occurs once in the first document
- This is used as the input by the LDA model

In [15]:
id2word[0]
print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]], end='')

'addition'

[[('addition', 1), ('body', 1), ('bring', 1), ('call', 1), ('car', 5), ('could', 1), ('day', 1), ('door', 2), ('early', 1), ('engine', 1), ('enlighten', 1), ('funky', 1), ('history', 1), ('host', 1), ('info', 1), ('know', 1), ('late', 1), ('lerxst', 1), ('line', 1), ('look', 2), ('mail', 1), ('make', 1), ('model', 1), ('name', 1), ('neighborhood', 1), ('nntp_poste', 1), ('park', 1), ('production', 1), ('really', 1), ('rest', 1), ('see', 1), ('separate', 1), ('small', 1), ('spec', 1), ('sport', 1), ('thank', 1), ('thing', 1), ('where', 1), ('wonder', 1), ('year', 1)]]

**Building the Topic Model**

- We have everything required to train the LDA model
- In addition to the corpus and dictionary, we need to provide the `number of topics` as well
- Apart from that, `alpha` and `eta` are hyperparameters that affect sparsity of the topics
- According to the Gensim docs, both defaults to 1.0/num_topics prior
- `chunksize` is the number of documents to be used in each training chunk
- `update_every` determines how often the model parameters should be updated
- `passes` is the total number of training passes

In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=20, random_state=100, update_every=1,
                                            chunksize=100, passes=10, alpha='auto',  per_word_topics=True)

In [17]:
# View the topics in LDA model
"""The above LDA model is built with 20 different topics where each topic is a combination of keywords and each keyword
contributes a certain weightage to the topic"""
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

'The above LDA model is built with 20 different topics where each topic is a combination of keywords and each keyword\ncontributes a certain weightage to the topic'

[(0,
  '0.175*"file" + 0.073*"entry" + 0.057*"error" + 0.053*"display" + '
  '0.040*"program" + 0.030*"sun" + 0.025*"version" + 0.024*"cool" + '
  '0.020*"output" + 0.020*"crash"'),
 (1,
  '0.032*"would" + 0.026*"say" + 0.022*"think" + 0.022*"people" + 0.020*"go" + '
  '0.018*"know" + 0.016*"make" + 0.016*"see" + 0.014*"come" + 0.013*"thing"'),
 (2,
  '0.091*"evidence" + 0.048*"book" + 0.041*"faith" + 0.037*"reason" + '
  '0.033*"exist" + 0.032*"claim" + 0.031*"religion" + 0.029*"believe" + '
  '0.027*"christian" + 0.023*"church"'),
 (3,
  '0.541*"ax" + 0.054*"car" + 0.028*"player" + 0.014*"engine" + 0.009*"dealer" '
  '+ 0.009*"mile" + 0.009*"expensive" + 0.009*"here" + 0.008*"extra" + '
  '0.008*"specify"'),
 (4,
  '0.087*"belief" + 0.084*"internet" + 0.079*"atheist" + 0.061*"distribution" '
  '+ 0.053*"wing" + 0.041*"atheism" + 0.038*"printer" + 0.034*"interface" + '
  '0.026*"multiple" + 0.020*"thinking"'),
 (5,
  '0.088*"normal" + 0.072*"port" + 0.071*"mouse" + 0.053*"hole" + 0.03

**Interpretation of the model output**

- Topic 0 is a represented as `0.175*"file" + 0.073*"entry" + 0.057*"error" + 0.053*"display" + 0.040*"program" + 0.030*"sun" + 0.025*"version" + 0.024*"cool" + 0.020*"output" + 0.020*"crash"`
- It means the top 10 keywords that contribute to this topic are: `'file'`, `'entry'`, `'error'` and so on and the weight of 'file' on topic 0 is 0.175
- The weights reflect how important a keyword is to that topic

**Compute Model Perplexity and Coherence Score**

In [18]:
# Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is - lower the better

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.486503603840534

Coherence Score:  0.49288861569721343


**Visualize the topics-keywords**

In [19]:
# The next step is to examine the produced topics and the associated keywords. There is no better tool than pyLDAvis package's
# interactive chart and is designed to work well with jupyter notebooks
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.335626 -0.217742       1        1  28.085712
18    -0.302087 -0.145044       2        1  18.153528
9     -0.271869 -0.004532       3        1  15.586514
19    -0.213919  0.101526       4        1   8.162916
3      0.014729  0.121690       5        1   5.748532
10    -0.144866  0.268453       6        1   5.679915
6     -0.129679  0.185631       7        1   5.134482
2     -0.061846 -0.076529       8        1   2.129818
14    -0.018009 -0.093240       9        1   1.930827
16     0.000236  0.059307      10        1   1.921904
11     0.069657  0.074448      11        1   1.423861
0      0.119143  0.020979      12        1   1.152395
12     0.109095 -0.008043      13        1   1.039403
8      0.085481 -0.055218      14        1   0.971923
7      0.131145 -0.010039      15        1   0.884548
4      0.160738 -0.037302      16        1   0.697290
13     0.172285 -0.042789      17        1   0.449756
15     0.198402 -0.044226      18        1   0.375347
5      0.200337 -0.048557      19        1   0.334445
17     0.216654 -0.048775      20        1   0.136887, topic_info=              Term          Freq         Total Category  logprob  loglift
3718            ax  36366.000000  36366.000000  Default  30.0000  30.0000
18            line  15682.000000  15682.000000  Default  29.0000  29.0000
197          write  13139.000000  13139.000000  Default  28.0000  28.0000
39            year   4567.000000   4567.000000  Default  27.0000  27.0000
140         people   7716.000000   7716.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
6226  redistribute      0.026008      1.250145  Topic20 -11.0266   2.7212
6951     judgement      0.026008      1.250163  Topic20 -11.0266   2.7211
9115      tomorrow      0.026008      1.250169  Topic20 -11.0266   2.7211
5664  announcement      0.026008      1.250196  Topic20 -11.0266   2.7211
4072          baby      0.026008      1.250173  Topic20 -11.0266   2.7211

[866 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1227      1  0.999079        accept
345      15  0.914019    acceptance
326      17  0.943324  accidentally
0         7  0.998122      addition
175       2  0.399800       address
...     ...       ...           ...
39        1  0.028244          year
39        3  0.824321          year
39       14  0.147349          year
722      13  0.992130     yesterday
570       1  0.998435         young

[1113 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 19, 10, 20, 4, 11, 7, 3, 15, 17, 12, 1, 13, 9, 8, 5, 14, 16, 6, 18])

**Plot Inference**

- Each bubble on the left-hand side plot represents a topic. The larger the bubble the more prevalent is that topic
- A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant
- A model with too many topics will typically have many overlaps, small sized bubbles clustered in one region of the chart
- Moving the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic

- We have seen Gensim’s inbuilt version of the LDA algorithm
- Mallet’s version, however, often gives a better quality of topics

**`We will improve upon this model by using Mallet’s version of LDA algorithm and then we will focus on how to arrive at the optimal number of topics given any large corpus of text`**


Gensim provides a wrapper to implement Mallet’s LDA from within Gensim itself. You only need to [download the zipfile](http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip), unzip it and provide the path to mallet in the unzipped directory to gensim.models.wrappers.LdaMallet

In [21]:
from gensim.models.wrappers import LdaMallet
os.environ.update({'MALLET_HOME':r'C:/Users/kalya/Python/mallet-2.0.8/mallet-2.0.8/'})

# Building LDA Mallet Model
mallet_path = r'C:/Users/kalya/Python/mallet-2.0.8/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command 'C:/Users/kalya/Python/mallet-2.0.8/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\kalya\AppData\Local\Temp\ecd75a_corpus.txt --output C:\Users\kalya\AppData\Local\Temp\ecd75a_corpus.mallet' returned non-zero exit status 1.

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

**Optimal number of topics for LDA**

- Build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value
- Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics
- Picking an even higher value can sometimes provide more granular sub-topics
- `If same keywords are being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large`
- The `compute_coherence_values()` trains multiple LDA models and provides the models and their corresponding coherence scores

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics
    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

"If the coherence score seems to keep increasing, it may make better sense to pick the model that gave the highest CV before flattening out. This is exactly the case here. Hencea bteer score would be 20"

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

**Refer this [link1](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/) and [link2](https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/) for more information**

[1. Topic modeling visualization](https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/)

[2. Finding the dominant topic](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)

[3. Gensim Tutorial](https://www.machinelearningplus.com/nlp/gensim-tutorial/#14howtotrainword2vecmodelusinggensim)

[4. Topic Modeling with Gensim](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)